<a href="https://colab.research.google.com/github/janinerottmann/Drillhole-Inspection/blob/master/Performance_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Performance Evaluation
The objective of this notebook is to find a suitable classifier for drill hole inspection. We compare the performance of a Support Vector machine (SVM), Random Forest (RF), Artificial Neural Network (ANN), K-Nearest-Neighbors (KNN) and Convolutional Neural Networks (CNN). The performace of each classifier is evaluated by Confusion Matrix, Balanced Accuracy, F1-Score and Log Loss. As features we use the Root Mean Square (RMS), maximum (MAX), crest-factor (CREST), kurtosis-factor (KURT) acceleration values in x, y and z direction.

In [ ]:
from google.colab import drive
import random
import pandas as pd
import numpy as np
from numpy import save, load
import torch
from scipy.signal import spectrogram, stft
import matplotlib.pyplot as plt

from sklearn.metrics import balanced_accuracy_score, f1_score, log_loss, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import Sequential
from tensorflow.python.keras.layers import concatenate
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Flatten, Conv1D, MaxPooling1D, GaussianNoise, Conv2D, BatchNormalization, Activation

In [ ]:
drive.mount('/content/drive')
!pwd
%cd ./drive/My\ Drive
datafolder = "data2/"

##1. Sample Dataset

In [ ]:
#choose class balance and step size
sampleSizeGood = 50
sampleSizeBad = 50
stepSize = 2
drillNumber = 1
#features = ['rmsAccX','rmsAccY','rmsAccZ']
features = ['rmsAccX','rmsAccY','rmsAccZ', 
            'maxAccX', 'maxAccY', 'maxAccZ', 
            'crestAccX', 'crestAccY', 'crestAccZ', 
            'kurtAccX', 'kurtAccY', 'kurtAccZ']
k = 5
batchSize = 10

In [ ]:
#quality file
file = datafolder + 'quality.csv'
quality = pd.read_csv(file)
quality = quality[['TeilNr', 'BohrlochNr', 'classifier', 'start_x', 'end_x', 'start_y', 'end_y']]
all_drills = quality

# get good and bad drills
goodDrills = quality[quality['classifier'] == False]
badDrills = quality[quality['classifier'] == True]

# choose random good and bad drills
random_good_drills = goodDrills.sample(n=sampleSizeGood)
random_bad_drills = badDrills.sample(n=sampleSizeBad)
all_drills = random_good_drills.append(random_bad_drills, ignore_index=True)

all_drills['datapoints_x']=all_drills['end_x'] - all_drills['start_x']
all_drills['datapoints_y']=all_drills['end_y'] - all_drills['start_y']

#identify datapoints
max_datapoints = all_drills.loc[:, ['datapoints_x', 'datapoints_y']].max().max()
Ninterpolate = max_datapoints * 1.1

#get filenames
files = []
parts = np.unique(all_drills['TeilNr'])

for part in parts:
  file = 'part' + str(int(part))+'.parq'
  files = np.append(files, file)

x_train_2D = []
x_train_3D = []
labels = []

for i in all_drills.index.values:
  
  #identify part and drill number of randomly selected drill
  row = all_drills[all_drills.index == i]
  part = int(row['TeilNr'])
  drill = int(row['BohrlochNr'])

  #open file
  file = 'part' + str(int(part))+'.parq'
  df = pd.read_parquet(datafolder + file)

  #locate sensor (v) and quality (q) data for each drill
  v = df[(df['TeilNr'] == part) & (df['BohrlochNr'] == drill) & (df['drill'] == drillNumber)]
  q = quality[(quality['TeilNr'] == part) & (quality['BohrlochNr'] == drill)]
  

  #get 2D-Tensor input
  xout = np.linspace(0,len(v),int(Ninterpolate))
  x_train_2D.append(np.concatenate([np.interp(xout, 
                                              np.arange(0,len(v)), 
                                              v[f].values).reshape(-1,1) for f in features], axis=0).reshape(1,-1))
  #get 3D-Tensor input
  xout = np.linspace(0,len(v),int(Ninterpolate))
  x_train_3D.append(np.concatenate([np.interp(xout, 
                                              np.arange(0,len(v)), 
                                              v[f].values).reshape(1,-1,1) for f in features], axis=2))
  
  #get Output
  labels.append(q.classifier)

x_2D = np.concatenate(x_train_2D,axis=0)
x_3D = np.concatenate(x_train_3D,axis=0)
y = np.concatenate(labels,axis=0)

In [ ]:
#optional save data to continue training later with the same datasampling
save('data/x_2D.npy', x_2D)
save('data/x_3D.npy', x_3D)
save('data/y.npy', y)

In [ ]:
#k fold cross validation
kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(x_2D, y):
    print("TRAIN:", train_index, "TEST:", test_index)

for train_index, test_index in kfold.split(x_3D, y):
    print("TRAIN:", train_index, "TEST:", test_index)

x_train_2D, x_test_2D, y_train_2D, y_test_2D = x_2D[train_index], x_2D[test_index], y[train_index], y[test_index]
x_train_3D, x_test_3D, y_train_3D, y_test_3D = x_3D[train_index], x_3D[test_index], y[train_index], y[test_index]

In [ ]:
#preprocessing
sc_x_2D = StandardScaler()
x_train_2D = sc_x_2D.fit_transform(x_train_2D)
x_test_2D = sc_x_2D.transform(x_test_2D)

sc_x_3D = StandardScaler()
x_train_3D = sc_x_3D.fit_transform(x_train_3D.reshape(-1, x_train_3D.shape[-1])).reshape(x_train_3D.shape)
x_test_3D = sc_x_3D.transform(x_test_3D.reshape(-1, x_test_3D.shape[-1])).reshape(x_test_3D.shape)

x_train_2D = preprocessing.normalize(x_train_2D)
x_test_2D = preprocessing.normalize(x_test_2D)

x_train_3D = preprocessing.normalize(x_train_3D.reshape(-1, x_train_3D.shape[-1])).reshape(x_train_3D.shape)
x_test_3D = preprocessing.normalize(x_test_3D.reshape(-1, x_test_3D.shape[-1])).reshape(x_test_3D.shape)

##2. Short Time Fourier Transformation

In [ ]:
#STFT
x_four = []
for bl in range(x_3D.shape[0]): # first dimension of tensor: drill holes
    tmpfft = [] ## zwischenspeicher für die verschiednen features
    for f in range(x_3D.shape[2]): #third dimension: features
        fft = np.abs(np.asarray(stft(x_3D[bl,:,f],fs=4096,nfft=256)[2])) # fs = Abtastrate in Herz
        tmpfft.append(fft.reshape(fft.shape[0],fft.shape[1],1)) 
    tmpfft = np.concatenate(tmpfft,axis=2) ## zu einer Matrix zusammenfügen
    x_four.append(tmpfft.reshape(1,tmpfft.shape[0],tmpfft.shape[1],tmpfft.shape[2]))
x_four = np.concatenate(x_four,axis=0)

x_four = 10. * np.log10(x_four+np.finfo(float).eps)

kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(x_four, y):
    print("TRAIN:", train_index, "TEST:", test_index)

x_train_four, x_test_four, y_train_four, y_test_four = x_four[train_index], x_four[test_index], y[train_index], y[test_index]

In [ ]:
#plot spectogram
nfft=256
fs=32000

neg_example = random.sample(list(np.where(y==1)[0]),1)[0]
pos_example = random.sample(list(np.where(y==0)[0]),1)[0]

In [ ]:
#pre- and finedrill
fig, axes = plt.subplots(2, 1, figsize=(16,8))
axes[0].plot(x_3D[pos_example])
axes[0].set_title('Positive example')
axes[1].plot(x_3D[neg_example])
axes[1].set_title('Negative example')  
_ = fig

In [ ]:
#pre- and finedrill as spectogram
fig, axes = plt.subplots(2, 1,figsize=(16,8))
axes[0].specgram(x_four[pos_example,:,-1].flatten(), Fs=fs, NFFT=nfft)
axes[0].set_title('Positive examples')
axes[1].specgram(x_four[neg_example,:,-1].flatten(), Fs=fs, NFFT=nfft)
axes[1].set_title('Negative examples')
_=fig

##3. Modelling

###3.1 SVM, ANN, RF, KNN

In [ ]:
class_weight = {0: 0.25, 1:0.75}

In [ ]:
RBF_SVC = SVC(class_weight=class_weight)
MLPClassifier = MLPClassifier()
RandomForestClassifier = RandomForestClassifier(class_weight=class_weight)
KNeighborsClassifier = KNeighborsClassifier()

classifiers = [RBF_SVC,
               MLPClassifier,
               RandomForestClassifier,
               KNeighborsClassifier]

names = ["RBF SVM",
         "Neural Net",
         "Random Forest",
         "Nearest Neighbors"]

In [ ]:
def algorithm_pipeline(x_train, y_train, model, param_grid, name):
    grid = GridSearchCV(estimator=model,
                        param_grid=param_grid, 
                        cv=2, 
                        n_jobs=-1, 
                        scoring='accuracy',
                        verbose=0)
    grid.fit(x_train, y_train)
    best_param = grid.best_params_
    return best_param

In [ ]:
#parameters for tuning

svm = {'C': [1, 2, 3],  # default = 1
       'gamma': [0.1, 0.01, 0.001, 0.0001]} #default = scale: 0.001

mlp = {'solver': ['sgd', 'adam', 'lbfgs'], #default = adam
       'alpha': [0.001, 0.01, 0.1], # default = 0.001
       'learning_rate' : ['constant', 'invscaling', 'adaptive'], #default = constant
       'max_iter': [150,200,250] #default = 200
       }

rf = {'n_estimators': [100, 150], # default = 100
      'min_samples_split': [1,2,3], # default = 2
      'min_samples_leaf': [1,2,3]} 

knn = {'n_neighbors': [3,5,7,10], #default = 5
       'leaf_size' : [10,20,30,40]} #default = 30

params = [svm, mlp, rf, knn]

In [ ]:
res_list = []

for model, name, param in zip(classifiers, names, params):

  #hyperparameter tuning
  best_params = algorithm_pipeline(x_train_2D, y_train_2D, model, param, name)
  for key, variable in best_params.items():
    setattr(model, key, variable)
  print(best_params)
  
  #fit
  model.fit(x_train_2D, y_train_2D)
  
  #predict
  y_pred = model.predict(x_test_2D)

  #metrics
  res =[name,
        balanced_accuracy_score(y_test_2D, y_pred)*100,
        precision_score(y_test_2D, y_pred)*100,
        recall_score(y_test_2D, y_pred)*100,
        f1_score(y_test_2D, y_pred)*100,
        log_loss(y_test_2D, y_pred),
        confusion_matrix(y_test_2D, y_pred)[0][0],
        confusion_matrix(y_test_2D, y_pred)[0][1],
        confusion_matrix(y_test_2D, y_pred)[1][0],
        confusion_matrix(y_test_2D, y_pred)[1][1],
        ]
  res_list.append(res)

In [ ]:
df = pd.DataFrame(res_list, columns= ["Model", 
                                      "Balanced Accuracy", 
                                      "Precision", 
                                      "Recall", 
                                      "F1", 
                                      "Log Loss", 
                                      "TN", #true negatives
                                      "FP", #false positives
                                      "FN", #false negatives
                                      "TP"  #true positives
                                      ]).round(decimals=2)

###3.2 CNN

In [ ]:
def CNN_model_four():
    model = Sequential()
    
    model.add(GaussianNoise( 0.01, input_shape=x_train_four.shape[-3:]))
    model.add(Conv2D(32, (7, 7), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(5, 5), padding = 'same'))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(64, (7, 7), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(4, 100), padding = 'same'))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
                  optimizer="adam",                       
                  metrics='accuracy')
    return model

In [ ]:
CNN_Four = CNN_model_four()

CNN_history_four = CNN_Four.fit(x_train_four, y_train_four, 
                                batch_size = batchSize, 
                                epochs=100, 
                                validation_data=(x_test_four,y_test_four),
                                verbose=0)

y_pred_four = CNN_Four.predict_classes(x_test_four, verbose=0, batch_size=batchSize)

res_CNN = ['CNN',
            balanced_accuracy_score(y_test_four, y_pred_four)*100,
            precision_score(y_test_four, y_pred_four)*100,
            recall_score(y_test_four, y_pred_four)*100,
            f1_score(y_test_four, y_pred_four)*100,
            log_loss(y_test_four, y_pred_four),
            confusion_matrix(y_test_four, y_pred_four)[0][0],
            confusion_matrix(y_test_four, y_pred_four)[0][1],
            confusion_matrix(y_test_four, y_pred_four)[1][0],
            confusion_matrix(y_test_four, y_pred_four)[1][1]]

In [ ]:
df_CNN = pd.DataFrame(res_CNN, index=["Model", 
                                      "Balanced Accuracy", 
                                      "Precision", 
                                      "Recall", 
                                      "F1", 
                                      "Log Loss", 
                                      "TP", #true positives
                                      "FP", #false positives
                                      "FN", #false negatives
                                      "TN"  #true negatives
                                       ]).T

##4. Results

In [ ]:
df = df.append(df_CNN)
df = df.sort_values(by=['Balanced Accuracy'], ascending=False)
df